In [1]:
import waveglow
import text
import audio
import torch
import utils
from nltk.tokenize import sent_tokenize
from src.config import TrainConfig, MelSpectrogramConfig, FastSpeechConfig
from src.modules import FastSpeech
import os
import nltk
from tqdm.notebook import tqdm
import numpy as np
from IPython import display

/workspace/local/razvitie/implementation/audio/stft.py:43: FutureWarning: Pass size=1024 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  fft_window = pad_center(fft_window, filter_length)
/workspace/local/razvitie/implementation/audio/stft.py:130: FutureWarning: Pass sr=22050, n_fft=1024, n_mels=80, fmin=0.0, fmax=8000.0 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mel_basis = librosa_mel_fn(


In [2]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In order to get wav from text, you need to specify the path to the text in the path_to_sents variable and the path to the model in the path_to_model variable. Then execute the script below. After executing the script, wav can be listened to by specifying the path in torch.Audio.

In [7]:
WaveGlow = utils.get_WaveGlow()
WaveGlow = WaveGlow.cuda()

path_to_sents = './test.txt'
path_to_model = '../FastSpeech2/model_new/checkpoint_30000.pth.tar'

train_config, model_config, mel_config = TrainConfig(), FastSpeechConfig(), MelSpectrogramConfig()
model = FastSpeech(model_config, mel_config)
model.load_state_dict(torch.load(path_to_model, map_location='cuda:0')['model'])
model = model.cuda()
model = model.eval()

/usr/local/lib/python3.8/dist-packages/torch/serialization.py:671: SourceChangeWarning: source code of class 'glow.WaveGlow' has changed. Saved a reverse patch to WaveGlow.patch. Run `patch -p0 < WaveGlow.patch` to revert your changes.
  warnings.warn(msg, SourceChangeWarning)
/usr/local/lib/python3.8/dist-packages/torch/serialization.py:671: SourceChangeWarning: source code of class 'torch.nn.modules.conv.ConvTranspose1d' has changed. Tried to save a patch, but couldn't create a writable file ConvTranspose1d.patch. Make sure it doesn't exist and your working directory is writable.
  warnings.warn(msg, SourceChangeWarning)
/usr/local/lib/python3.8/dist-packages/torch/serialization.py:671: SourceChangeWarning: source code of class 'torch.nn.modules.container.ModuleList' has changed. Saved a reverse patch to ModuleList.patch. Run `patch -p0 < ModuleList.patch` to revert your changes.
  warnings.warn(msg, SourceChangeWarning)
/usr/local/lib/python3.8/dist-packages/torch/serialization.py:6

In [8]:
def synthesis(model, text, train_config, alpha=1.0):
    text = np.array(text)
    text = np.stack([text])
    src_pos = np.array([i+1 for i in range(text.shape[1])])
    src_pos = np.stack([src_pos])
    sequence = torch.from_numpy(text).long().to(train_config.device)
    src_pos = torch.from_numpy(src_pos).long().to(train_config.device)
    
    with torch.no_grad():
        mel = model.forward(sequence, src_pos, alpha=alpha, train_config=train_config)
    return mel[0].cpu().transpose(0, 1), mel.contiguous().transpose(1, 2)


def get_data(path_to_sents, train_config):
    with open(path_to_sents, 'r') as f:
        _text = f.read()
        
    sents = sent_tokenize(_text)
    data_list = list(text.text_to_sequence(sent, train_config.text_cleaners) for sent in sents)

    return data_list

def inference(model, path_to_sents, train_config):
    data_list = get_data(path_to_sents, train_config)
    for speed in [0.8, 1., 1.3]:
        for i, phn in tqdm(enumerate(data_list)):
            mel, mel_cuda = synthesis(model, phn, train_config, speed)

            os.makedirs("results", exist_ok=True)

            audio.tools.inv_mel_spec(
                mel, f"results/s={speed}_{i}.wav"
            )

            waveglow.inference.inference(
                mel_cuda, WaveGlow,
            f"results/s={speed}_{i}_waveglow.wav"
        )

In [9]:
inference(model, path_to_sents, train_config)

0it [00:00, ?it/s]

/workspace/local/razvitie/implementation/audio/audio_processing.py:52: FutureWarning: Pass size=1024 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  win_sq = librosa_util.pad_center(win_sq, n_fft)


0it [00:00, ?it/s]

0it [00:00, ?it/s]

In [12]:
display.Audio("results/s=1.0_0_waveglow.wav")